In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import datetime

In [ ]:
dataset = pd.read_csv('weblog.csv')
dataset.head()

In [ ]:
dataset.iloc[0:1,0]

In [ ]:
df = dataset.iloc[:580,:]
df.head()

In [ ]:
data = {'IP_Address':['10.128.2.1'], 'Time_Difference_Mean':[99], 'Time_Difference_Variance':[54], 'Time_Difference_Sum':[105], 'Time_Difference_Maximum':[15], 'Character-bigrams':[45], 'Character-trigrams':[34], 'Character-ngrams':[74], 'Count_of_most_visited_page':[14], 'Status':[200], 'Number_of_records':[40]}
work_dataset = pd.DataFrame(data)
work_dataset

In [ ]:
def time_stats(dataframe):
    time_array = dataframe.iloc[:, 1].values
    max_time_diff = 0
    count = 0
    sum_time_diff = 0
    for i in range(1, len(time_array)):
        time1 = time_array[i]
        time2 = time_array[i-1]
        index1 = 0
        for char in time1:
            if(char==':'):
                break
            index1 = index1 +1
        time1 = time1[index1+1:]
        index2 = 0
        for char in time2:
            if(char==':'):
                break
            index2 = index2 + 1
        time2 = time2[index2+1:]
        dtime1 = datetime.datetime.strptime(time1, '%H:%M:%S')
        dtime2 = datetime.datetime.strptime(time2, '%H:%M:%S')
        diff = (dtime1 - dtime2).seconds
        sum_time_diff = sum_time_diff + diff
        count = count + 1
        
        if max_time_diff < diff:
            max_time_diff = diff
    
    mean_time_diff = 0
    if count!=0:
        mean_time_diff = sum_time_diff/count
    
    var_sum = 0
    for i in range(1, len(time_array)):
        time1 = time_array[i]
        time2 = time_array[i-1]
        index1 = 0
        for char in time1:
            if(char==':'):
                break
            index1 = index1 +1
        time1 = time1[index1+1:]
        index2 = 0
        for char in time2:
            if(char==':'):
                break
            index2 = index2 + 1
        time2 = time2[index2+1:]
        dtime1 = datetime.datetime.strptime(time1, '%H:%M:%S')
        dtime2 = datetime.datetime.strptime(time2, '%H:%M:%S')
        diff = (dtime1 - dtime2).seconds
        curvar = (mean_time_diff-diff)*(mean_time_diff-diff)
        var_sum = var_sum + curvar
    
    variance_time_diff = 0
    if count != 0:
        variance_time_diff = var_sum/count
    return max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff

In [ ]:
time_stats(df)

In [ ]:
import re
import string
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams

In [ ]:
def most_frequent(List): 
    return max(set(List), key = List.count)

In [ ]:
def bigram_stats(dataframe):
    url_array = dataframe.iloc[:,2].values
    full_string = ""
    for url in url_array:
        full_string = full_string + url
    full_string = full_string.lower()
    full_string = re.sub(r'\d+', '', full_string)
    table = str.maketrans({key: None for key in string.punctuation})
    full_string = full_string.translate(table) 
    full_string = full_string.strip()
    full_string_tokens = word_tokenize(full_string)
    for char in full_string_tokens:
        if char == 'get':
            full_string_tokens.remove('get')
    for char in full_string_tokens:
        if char == 'httpget':
            full_string_tokens.remove('httpget')
    for char in full_string_tokens:
        if char == 'httppost':
            full_string_tokens.remove('httppost')
    for char in full_string_tokens:
        if char == 'post':
            full_string_tokens.remove('post')
    
    count_most_visited_page = 0
    if len(full_string_tokens)!=0:
        most_visited_page = most_frequent(full_string_tokens)
        count_most_visited_page = full_string_tokens.count(most_visited_page)
        
    
    count_most_appearing_bigram = 0
    bigrams = list(nltk.bigrams(full_string_tokens))
    if len(bigrams)!=0:
        most_appearing_bigram = most_frequent(bigrams)
        count_most_appearing_bigram = bigrams.count(most_appearing_bigram)
    
    count_most_appearing_trigram = 0
    trigrams = list(nltk.trigrams(full_string_tokens))
    if len(trigrams)!=0:
        most_appearing_trigram = most_frequent(trigrams)
        count_most_appearing_trigram = trigrams.count(most_appearing_trigram)
    
    count_most_appearing_ngram = 0
    ngrams = list(nltk.ngrams(full_string_tokens, 6))
    if len(ngrams)!=0:
        most_appearing_ngram = most_frequent(ngrams)
        count_most_appearing_ngram = ngrams.count(most_appearing_ngram)
        
    return count_most_visited_page, count_most_appearing_bigram, count_most_appearing_trigram, count_most_appearing_ngram

In [ ]:
bigram_stats(df)

In [ ]:
from collections import Counter

In [ ]:
def most_visited_ip(dataframe):
    ip = dataframe.iloc[:,0].values
    x = Counter(ip)
    return x.most_common(1)[0][0]

In [ ]:
most_visited_ip(df)

In [ ]:
def most_freq_status(dataframe):
    status = dataframe.iloc[:,3].values
    x = Counter(status)
    return x.most_common(1)[0][0]

In [ ]:
most_freq_status(df)

In [ ]:
startindex = 0
endindex = 0
prevdate = ""
p = 1
for i,row in dataset.iterrows():
    
    date = row['Time'][1:12]
    if i==0:
        prevdate = date
    else:
        if date == prevdate:
            endindex = endindex + 1
        else:
            data = dataset.iloc[startindex:endindex+1, :]
            max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff = time_stats(data)
            #print(max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff)
            count_most_visited_page, count_most_appearing_bigram, count_most_appearing_trigram, count_most_appearing_ngram = bigram_stats(data)
            #print(count_most_visited_page, count_most_appearing_bigram, count_most_appearing_trigram, count_most_appearing_ngram)
            most_vis_ip = most_visited_ip(data)
            most_frequent_status = most_freq_status(data)
            #print(most_vis_ip, most_frequent_status, end = ' ')
            #print()
            if most_vis_ip == None:
                most_vis_ip = '10.130.2.1'
            if most_frequent_status == None:
                most_frequent_status = 200
            work_dataset.loc[p]=[count_most_appearing_bigram, count_most_appearing_ngram, count_most_appearing_trigram, count_most_visited_page, most_vis_ip, endindex - startindex + 1, most_frequent_status, max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff]
            startindex = endindex
            #endindex = endindex + 1
            prevdate = date
            p = p + 1
            #print(max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff)

In [ ]:
work_dataset

In [ ]:
Ip_rep = np.array([])
for i,row in work_dataset.iterrows():
    ip = row[4]
    ip = ip.replace('.', '')
    a = int(ip)
    Ip_rep = np.append(Ip_rep, a)
Ip_rep
work_dataset['IP_rep'] = Ip_rep

In [ ]:
work_dataset

In [ ]:
work_dataset.drop('IP_Address', axis=1, inplace=True)

In [ ]:
work_dataset

In [ ]:
X = work_dataset.iloc[:,:].values

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)
X

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 5,random_state = 0)

In [ ]:
kmeans.fit(X)

In [ ]:
y_kmeans = kmeans.predict(X)
y_kmeans

In [ ]:
plt.scatter(X[y_kmeans == 0, 4], X[y_kmeans == 0, 3], c = 'red', label = 'Cluster 1')
plt.scatter(X[y_kmeans == 1, 4], X[y_kmeans == 1, 3], c = 'blue', label = 'Cluster 2')
plt.scatter(X[y_kmeans == 2, 4], X[y_kmeans == 2, 3], c = 'green', label = 'Cluster 3')
plt.scatter(X[y_kmeans == 3, 4], X[y_kmeans == 3, 3], c = 'cyan', label = 'Cluster 4')
plt.scatter(X[y_kmeans == 4, 4], X[y_kmeans == 4, 3], c = 'magenta', label = 'Cluster 5')
#plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'yellow', label = 'Centroids')
plt.title('K_means_Clustering')
plt.xlabel('Number_of_records')
plt.ylabel('Count_of_most_visited_pages')
plt.legend()
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
cor = work_dataset.corr()
sns.heatmap(cor, square = True)
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering, AffinityPropagation 

In [ ]:
model = AgglomerativeClustering(n_clusters=5, affinity = 'euclidean', linkage = 'ward')
clust_labels1 = model.fit_predict(X)
clust_labels1

In [ ]:
agglomerative = pd.DataFrame(clust_labels1)

In [ ]:
plt.scatter(X[clust_labels1 == 0, 4], X[clust_labels1 == 0, 3], c = 'red', label = 'Cluster 1')
plt.scatter(X[clust_labels1 == 1, 4], X[clust_labels1 == 1, 3], c = 'blue', label = 'Cluster 2')
plt.scatter(X[clust_labels1 == 2, 4], X[clust_labels1 == 2, 3], c = 'green', label = 'Cluster 3')
plt.scatter(X[clust_labels1 == 3, 4], X[clust_labels1 == 3, 3], c = 'cyan', label = 'Cluster 4')
plt.scatter(X[clust_labels1 == 4, 4], X[clust_labels1 == 4, 3], c = 'magenta', label = 'Cluster 5')
plt.title('Agglomerative Clustering')
plt.xlabel('Number_of_records')
plt.ylabel('Count_of_most_visited_pages')
plt.legend()
plt.show()

In [ ]:
model_affinity = AffinityPropagation(damping = 0.5, max_iter = 250, affinity = 'euclidean')
model_affinity.fit(X)
clust_labels2 = model_affinity.predict(X)
cent2 = model_affinity.cluster_centers_
affinity = pd.DataFrame(clust_labels2)

In [ ]:
plt.scatter(X[clust_labels2 == 0, 4], X[clust_labels2 == 0, 3], c = 'red', label = 'Cluster 1')
plt.scatter(X[clust_labels2 == 1, 4], X[clust_labels2 == 1, 3], c = 'blue', label = 'Cluster 2')
plt.scatter(X[clust_labels2 == 2, 4], X[clust_labels2 == 2, 3], c = 'green', label = 'Cluster 3')
plt.scatter(X[clust_labels2 == 3, 4], X[clust_labels2 == 3, 3], c = 'cyan', label = 'Cluster 4')
plt.scatter(X[clust_labels2 == 4, 4], X[clust_labels2 == 4, 3], c = 'magenta', label = 'Cluster 5')
plt.title('Affinity Propagation Clustering')
plt.xlabel('Number_of_records')
plt.ylabel('Count_of_most_visited_pages')
plt.legend()
plt.show()

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
clustering = SpectralClustering(n_clusters=5, assign_labels="discretize", random_state=0).fit(X)
clust_labels3 = clustering.labels_

In [ ]:
clust_labels3

In [ ]:
plt.scatter(X[clust_labels3 == 0, 4], X[clust_labels3 == 0, 3], c = 'red', label = 'Cluster 1')
plt.scatter(X[clust_labels3 == 1, 4], X[clust_labels3 == 1, 3], c = 'blue', label = 'Cluster 2')
plt.scatter(X[clust_labels3 == 2, 4], X[clust_labels3 == 2, 3], c = 'green', label = 'Cluster 3')
plt.scatter(X[clust_labels3 == 3, 4], X[clust_labels3 == 3, 3], c = 'cyan', label = 'Cluster 4')
plt.scatter(X[clust_labels3 == 4, 4], X[clust_labels3 == 4, 3], c = 'magenta', label = 'Cluster 5')
plt.title('Spectral Clustering')
plt.xlabel('Number_of_records')
plt.ylabel('Count_of_most_visited_pages')
plt.legend()
plt.show()